In [21]:
import yaml
import pyodbc
from market_growth_analysis.etl.stagging import *
import pandas as pd
import numpy as np
from datetime import datetime
# Load the YAML file
with open('../../conf/global.yml', 'r') as f:
    columns = yaml.safe_load(f)

financial = pd.read_csv("../../data/processed_03/financial-prices-ratios.csv", index_col=0)

# Clean Ratios

In [22]:
columns_ratios = columns['columns_ratios']
columns_ratios_all = columns['column_id'] + columns['columns_ratios']
ratios_df = financial[columns_ratios_all]

In [23]:
for column in columns_ratios:
    alpha = 0.1
    top_percentile = 1 - (alpha/2)
    low_percentile = alpha/2

    # Calculate the alpha percentile value
    top_percentile_value = ratios_df[column].quantile(top_percentile)
    low_percentile_value = ratios_df[column].quantile(low_percentile)

    # Replace outliers with percentiles value
    ratios_df[column] = ratios_df[column].where(ratios_df[column] <= top_percentile_value, top_percentile_value)
    ratios_df[column] = ratios_df[column].where(ratios_df[column] >= low_percentile_value, low_percentile_value)

C:\Users\germa\AppData\Local\Temp/ipykernel_19084/2285555845.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df[column] = ratios_df[column].where(ratios_df[column] <= top_percentile_value, top_percentile_value)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/2285555845.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df[column] = ratios_df[column].where(ratios_df[column] >= low_percentile_value, low_percentile_value)


In [24]:
# # Replace -inf and inf with None
# ratios_df = ratios_df.replace([-np.inf, np.inf], None)

# Make sure types are correct
ratios_df[columns['columns_ratios']] = ratios_df[columns['columns_ratios']].astype(float)
ratios_df['PK'] = ratios_df['PK'].astype(int)
ratios_df['Date'] = pd.to_datetime(ratios_df['Date'])



C:\Users\germa\AppData\Local\Temp/ipykernel_19084/3579048468.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df[columns['columns_ratios']] = ratios_df[columns['columns_ratios']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/3579048468.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df['PK'] = ratios_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/3579048468.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [25]:
ratios_df.describe()

,PK,Enterprise Value (EV),EV / Revenue,EV / EBITDA,EV / EBIT,EV / Invested Capital,Free Cash Flow (FCF),EV / Free Cash Flow,EV / FCF,P/E,P/S,P/CF
count,48682.000000,48682.000000,48682.000000,48682.000000,48682.000000,48682.000000,48682.000000,48682.000000,48682.000000,48682.000000,48682.000000,48682.000000
mean,24340.500000,12331.989866,8.852364,13.536706,18.247282,524.189009,172.850957,10.302188,10.302188,11.283680,5.760072,7.586840
std,14053.427239,24023.533394,10.287518,24.607281,38.217054,384.378032,645.125712,57.329444,57.329444,28.562434,8.336052,18.632825
min,0.000000,19.011910,0.521202,-48.383906,-75.231102,1.233395,-862.528500,-120.391322,-120.391322,-52.223474,0.248554,-38.373482
25%,12170.250000,345.439511,1.558797,1.746754,-2.097587,29.004605,-51.938000,-13.121508,-13.121508,-3.065383,0.915666,-0.634467
50%,24340.500000,1895.805545,3.670317,12.307924,15.996976,833.265745,1.654500,4.961607,4.961607,11.739540,2.172671,7.918946
75%,36510.750000,9290.116096,13.021990,23.056843,34.653750,833.265745,157.609750,30.765275,30.765275,22.735935,5.404216,15.665535
max,48681.000000,89951.591328,31.328791,53.677262,80.984949,833.265745,1942.364500,133.024080,133.024080,81.249846,27.258971,47.650022


In [26]:
ratios_df.head(2)

,PK,ticker,Date,Enterprise Value (EV),EV / Revenue,EV / EBITDA,EV / EBIT,EV / Invested Capital,Free Cash Flow (FCF),EV / Free Cash Flow,EV / FCF,P/E,P/S,P/CF
0,0,A,2009-10-31,8728.061863,1.947793,41.761062,80.984949,53.546392,394.0,22.152441,22.152441,-52.223474,1.366450,15.007505
1,1,A,2010-10-31,11048.124516,2.029413,14.385579,19.519655,77.803694,-456.0,-24.228343,-24.228343,12.831291,1.614093,12.238335


# Clean Income Statement

In [27]:
columns_income_statement = columns['column_id'] + columns['columns_income_statement']
income_statememnt_df = financial[columns_income_statement]

In [28]:
income_statememnt_df.describe()

,PK,Revenue,Cost Of Goods Sold,Gross Profit,Research And Development Expenses,SG&A Expenses,Other Operating Income Or Expenses,Operating Expenses,Operating Income,Total Non-Operating Income/Expense,...,Other Income,Income From Continuous Operations,Income From Discontinued Operations,Net Income,EBITDA,EBIT,Basic Shares Outstanding,Shares Outstanding,Basic EPS,EPS - Earnings Per Share
count,48682.000000,45830.000000,41548.000000,45863.000000,18651.000000,46769.000000,21011.000000,41545.000000,48509.000000,43471.000000,...,1266.000000,48582.000000,8475.000000,48582.000000,43742.000000,43739.000000,48481.000000,48453.000000,4.849000e+04,4.868200e+04
mean,24340.500000,5828.984627,3844.764539,2341.758908,318.564431,1097.647272,-349.762005,1671.327548,684.502367,-89.600865,...,9.991779,455.223731,59.031875,440.014086,1003.163644,646.244519,261.902189,267.218536,-4.866593e+03,-4.848221e+03
std,14053.427239,22095.085749,16770.040250,8326.179440,1606.015274,4518.301051,2567.714626,5993.031644,3186.900852,858.858701,...,151.357418,2371.575602,1030.487561,2370.837776,4140.744245,3106.883899,2043.169206,2070.625319,7.404428e+05,7.389811e+05
min,0.000000,-8219.000000,-13697.000000,-41649.440000,-18.936000,-6231.506000,-124834.000000,-43812.160000,-30363.000000,-59155.360000,...,-371.410000,-23251.000000,-8368.000000,-22802.000000,-24504.400000,-30363.000000,-2278.226000,-2278.226000,-1.595500e+08,-1.595500e+08
25%,12170.250000,97.328300,41.587000,46.837500,6.356000,23.177000,-71.056000,51.726000,-5.706000,-52.646000,...,0.000000,-10.282750,-2.291500,-11.168000,-1.870425,-8.328700,15.057000,15.273000,-5.600000e-01,-5.500000e-01
50%,24340.500000,597.351000,327.696000,253.900000,28.694000,104.156000,-6.043000,213.356000,37.975000,-4.338000,...,0.000000,20.320000,0.000000,19.326000,76.144950,37.200100,44.757000,45.589000,6.497000e-01,6.200000e-01
75%,36510.750000,2958.130250,1840.637750,1134.173000,99.948500,480.728000,0.000000,831.445000,289.900100,0.176000,...,0.000000,182.291000,8.907000,175.122875,485.000000,298.891850,135.445000,138.321000,2.180000e+00,2.136875e+00
max,48681.000000,611289.000000,463721.000000,225152.000000,73213.000000,138428.000000,43812.160000,212904.000000,119437.000000,31744.110000,...,4044.055000,99803.000000,76486.910000,99803.000000,130541.000000,119437.000000,373456.000000,373456.000000,2.089355e+05,1.771514e+05


In [29]:
# Make sure types are correct
income_statememnt_df[columns['columns_income_statement']] = income_statememnt_df[columns['columns_income_statement']].astype(float)
income_statememnt_df['PK'] = income_statememnt_df['PK'].astype(int)
income_statememnt_df['Date'] = pd.to_datetime(income_statememnt_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_19084/800232268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_statememnt_df[columns['columns_income_statement']] = income_statememnt_df[columns['columns_income_statement']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/800232268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_statememnt_df['PK'] = income_statememnt_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/800232268.py:4: SettingWithCopyWarning

In [30]:
income_statememnt_df.describe()

,PK,Revenue,Cost Of Goods Sold,Gross Profit,Research And Development Expenses,SG&A Expenses,Other Operating Income Or Expenses,Operating Expenses,Operating Income,Total Non-Operating Income/Expense,...,Other Income,Income From Continuous Operations,Income From Discontinued Operations,Net Income,EBITDA,EBIT,Basic Shares Outstanding,Shares Outstanding,Basic EPS,EPS - Earnings Per Share
count,48682.000000,45830.000000,41548.000000,45863.000000,18651.000000,46769.000000,21011.000000,41545.000000,48509.000000,43471.000000,...,1266.000000,48582.000000,8475.000000,48582.000000,43742.000000,43739.000000,48481.000000,48453.000000,4.849000e+04,4.868200e+04
mean,24340.500000,5828.984627,3844.764539,2341.758908,318.564431,1097.647272,-349.762005,1671.327548,684.502367,-89.600865,...,9.991779,455.223731,59.031875,440.014086,1003.163644,646.244519,261.902189,267.218536,-4.866593e+03,-4.848221e+03
std,14053.427239,22095.085749,16770.040250,8326.179440,1606.015274,4518.301051,2567.714626,5993.031644,3186.900852,858.858701,...,151.357418,2371.575602,1030.487561,2370.837776,4140.744245,3106.883899,2043.169206,2070.625319,7.404428e+05,7.389811e+05
min,0.000000,-8219.000000,-13697.000000,-41649.440000,-18.936000,-6231.506000,-124834.000000,-43812.160000,-30363.000000,-59155.360000,...,-371.410000,-23251.000000,-8368.000000,-22802.000000,-24504.400000,-30363.000000,-2278.226000,-2278.226000,-1.595500e+08,-1.595500e+08
25%,12170.250000,97.328300,41.587000,46.837500,6.356000,23.177000,-71.056000,51.726000,-5.706000,-52.646000,...,0.000000,-10.282750,-2.291500,-11.168000,-1.870425,-8.328700,15.057000,15.273000,-5.600000e-01,-5.500000e-01
50%,24340.500000,597.351000,327.696000,253.900000,28.694000,104.156000,-6.043000,213.356000,37.975000,-4.338000,...,0.000000,20.320000,0.000000,19.326000,76.144950,37.200100,44.757000,45.589000,6.497000e-01,6.200000e-01
75%,36510.750000,2958.130250,1840.637750,1134.173000,99.948500,480.728000,0.000000,831.445000,289.900100,0.176000,...,0.000000,182.291000,8.907000,175.122875,485.000000,298.891850,135.445000,138.321000,2.180000e+00,2.136875e+00
max,48681.000000,611289.000000,463721.000000,225152.000000,73213.000000,138428.000000,43812.160000,212904.000000,119437.000000,31744.110000,...,4044.055000,99803.000000,76486.910000,99803.000000,130541.000000,119437.000000,373456.000000,373456.000000,2.089355e+05,1.771514e+05


# Clean balance sheet

In [31]:
columns_balance_sheet = columns['column_id'] + columns['columns_balance_sheet']
balance_sheet_df = financial[columns_balance_sheet]

In [32]:
# Make sure types are correct
balance_sheet_df[columns['columns_balance_sheet']] = balance_sheet_df[columns['columns_balance_sheet']].astype(float)
balance_sheet_df['PK'] = balance_sheet_df['PK'].astype(int)
balance_sheet_df['Date'] = pd.to_datetime(balance_sheet_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_19084/19496767.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_df[columns['columns_balance_sheet']] = balance_sheet_df[columns['columns_balance_sheet']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/19496767.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_df['PK'] = balance_sheet_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/19496767.py:4: SettingWithCopyWarning: 
A value is trying to b

# Clean columns_cash_flow_statement

In [33]:
columns_cash_flow_statement = columns['column_id'] + columns['columns_cash_flow_statement']
cash_flow_statement_df = financial[columns_cash_flow_statement]

In [34]:
# Make sure types are correct
cash_flow_statement_df[columns['columns_cash_flow_statement']] = cash_flow_statement_df[columns['columns_cash_flow_statement']].astype(float)
cash_flow_statement_df['PK'] = cash_flow_statement_df['PK'].astype(int)
cash_flow_statement_df['Date'] = pd.to_datetime(cash_flow_statement_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_19084/475317979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cash_flow_statement_df[columns['columns_cash_flow_statement']] = cash_flow_statement_df[columns['columns_cash_flow_statement']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/475317979.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cash_flow_statement_df['PK'] = cash_flow_statement_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/475317979.py:4: SettingW

# Clean prices

In [35]:
columns_prices = columns['column_id'] + columns['columns_prices']
prices_df = financial[columns_prices]

In [36]:
prices_df.describe()

,PK,longevity,Close,Volume,Growth -1,Growth +1,Growth +5,avgGrowth -10,avgGrowth -5
count,48682.000000,48682.000000,4.868200e+04,4.868200e+04,45784.000000,43300.000000,25913.000000,30188.000000,37200.000000
mean,24340.500000,16.357298,6.773635e+07,3.117842e+07,0.152046,0.131056,0.743536,0.166655,0.152540
std,14053.427239,12.939564,8.627875e+09,1.609766e+08,1.655202,1.494885,3.235625,0.439694,0.657748
min,0.000000,0.000000,2.000000e-02,0.000000e+00,-0.999999,-0.999999,-1.000000,-0.658006,-0.856566
25%,12170.250000,5.000000,9.800000e+00,1.078825e+06,-0.216013,-0.217619,-0.280234,0.048131,-0.004476
50%,24340.500000,14.000000,2.220000e+01,5.727600e+06,0.035520,0.031640,0.274585,0.118925,0.098110
75%,36510.750000,25.000000,4.773000e+01,2.124897e+07,0.298172,0.288597,1.036866,0.211361,0.221485
max,48681.000000,61.000000,1.373400e+12,1.197745e+10,198.999997,198.999997,278.416663,27.979160,56.027803


In [37]:
# Make sure types are correct
prices_df[columns['columns_prices']] = prices_df[columns['columns_prices']].astype(float)
prices_df[['PK','longevity']] = prices_df[['PK','longevity']].astype(int)
prices_df['Date'] = pd.to_datetime(prices_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_19084/3747507719.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df[columns['columns_prices']] = prices_df[columns['columns_prices']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/3747507719.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df[['PK','longevity']] = prices_df[['PK','longevity']].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_19084/3747507719.py:4: SettingWithCopyWarning: 
A value is trying to be set on

# Create Dim table

In [38]:
columns_dim = columns['columns_dim']
dim_df = financial[columns_dim]

In [39]:
dim_grouped_df = dim_df.groupby('ticker').first().reset_index()

# Save dataframes to load

In [40]:
ratios_df.to_csv("../../data/load_04/ratios_df.csv")
income_statememnt_df.to_csv("../../data/load_04/income_statememnt_df.csv")
balance_sheet_df.to_csv("../../data/load_04/balance_sheet_df.csv")
cash_flow_statement_df.to_csv("../../data/load_04/cash_flow_statement_df.csv")
prices_df.to_csv("../../data/load_04/prices_df.csv")
dim_grouped_df.to_csv("../../data/load_04/dim_grouped_df.csv")